In [1]:
# Celda 1: Test Integral del Sistema Bodega (CORREGIDO)
import os
import sys
from pathlib import Path
import json
from pprint import pprint

# 1. CONFIGURACIÓN Y DIAGNÓSTICO INICIAL
print("=== TEST INTEGRAL SISTEMA BODEGA ===\n")

# Detectar ubicación y configurar rutas
current = Path().resolve()
project_root = current.parent.parent
print(f"🔍 Ubicación: {current}")
print(f"📂 Proyecto raíz: {project_root}")

# Añadir path Python
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print("✅ Path añadido a Python")

# 2. VERIFICAR ARCHIVOS ESENCIALES
print("\n2. 📁 VERIFICACIÓN DE ESTRUCTURA:")
essential_files = {
    'Configuración': ['.env', 'config/settings.yaml'],
    'Utilidades': ['utils/env_loader.py', 'utils/db_system_detector.py'],
    'Proveedores': ['providers/provider_manager.py', 'providers/base_provider.py',
                   'providers/llm/openai_provider.py', 'providers/llm/claude_provider.py'],
    'Agentes': ['agent_modules/base_agent.py', 'agent_modules/db_agent/agent.py']
}

missing_files = []
for category, files in essential_files.items():
    print(f"\n▸ {category}:")
    for file in files:
        file_path = project_root / file
        status = "✅" if file_path.exists() else "❌"
        print(f"  {status} {file}")
        if not file_path.exists():
            missing_files.append(file)

if missing_files:
    print(f"\n⚠️ ARCHIVOS FALTANTES: {len(missing_files)}")
else:
    print("\n✅ ESTRUCTURA COMPLETA")

# 3. CARGAR VARIABLES DE ENTORNO
print("\n3. 🔐 CARGA DE VARIABLES DE ENTORNO:")
try:
    from dotenv import load_dotenv
    env_path = project_root / ".env"
    load_dotenv(env_path)
    
    # Verificar variables MySQL
    mysql_vars = {
        'MYSQL_HOST': os.getenv('MYSQL_HOST'),
        'MYSQL_PORT': os.getenv('MYSQL_PORT'),
        'MYSQL_USER': os.getenv('MYSQL_USER'),
        'MYSQL_DATABASE': os.getenv('MYSQL_DATABASE'),
        'MYSQL_PASSWORD': os.getenv('MYSQL_PASSWORD')
    }
    
    print("\n▸ Configuración MySQL:")
    for key, value in mysql_vars.items():
        if value:
            display_value = value if 'PASSWORD' not in key else '********'
            print(f"  ✓ {key}: {display_value}")
        else:
            print(f"  ⚠ {key}: No definido")
except Exception as e:
    print(f"❌ Error: {e}")

# 4. INICIALIZAR PROVIDER MANAGER
print("\n4. 🚀 INICIALIZACIÓN DEL SISTEMA:")
try:
    from providers.provider_manager import ProviderManager
    from agent_modules.db_agent.agent import DBAgent
    
    config_path = project_root / "config" / "settings.yaml"
    manager = ProviderManager(str(config_path))
    print(f"✅ ProviderManager: {manager.get_current_provider_name()}")
    
    # Crear DBAgent
    db_agent = DBAgent(provider_manager=manager)
    print("✅ DBAgent creado")
    
except Exception as e:
    print(f"❌ Error: {e}")

# 5. CONECTAR A BASE DE DATOS
print("\n5. 🔌 TEST DE CONEXIÓN A BASE DE DATOS:")
test_connection = {
    "type": "mysql",
    "host": os.getenv('MYSQL_HOST', '127.0.0.1'),
    "port": int(os.getenv('MYSQL_PORT', 3307)),
    "user": os.getenv('MYSQL_USER', 'root'),
    "password": os.getenv('MYSQL_PASSWORD', ''),
    "database": os.getenv('MYSQL_DATABASE', 'bodega')
}

# Test básico de conexión
result = db_agent.run({
    "prompt": "SELECT DATABASE() as database_actual, VERSION() as mysql_version",
    "connection": test_connection
})

if "error" not in result:
    db_actual, version = result.get('results', [[None, None]])[0]
    print(f"✅ MySQL Version: {version}")
    print(f"✅ Database: {db_actual}")
else:
    print(f"❌ Error conexión: {result.get('error')}")

# 6. DETECTAR SISTEMA DE BASES DE DATOS
print("\n6. 🔎 DETECCIÓN DE SISTEMA:")
try:
    from utils.db_system_detector import run as detect_system
    system_info = detect_system()
    
    print("▸ Sistema operativo:", system_info['environment']['system'])
    print("▸ Arquitectura:", system_info['environment']['architecture'])
    print("▸ Bases de datos detectadas:")
    for db_name, info in system_info['database_systems'].items():
        status = "✅" if info['status'] == 'fully_available' else "⚠️"
        print(f"  {status} {db_name}: {info['status']}")
        if info['ports_detected']:
            print(f"     Puertos: {info['ports_detected']}")
except Exception as e:
    print(f"❌ Error: {e}")

# 7. VERIFICAR ESTRUCTURA DE BODEGA
print("\n7. 📊 ESTRUCTURA DE BASE DE DATOS BODEGA:")
result = db_agent.run({
    "prompt": "SHOW TABLES",
    "connection": test_connection
})

if "error" not in result:
    tablas = [t[0] for t in result.get('results', [])]
    print(f"✅ Total de tablas: {len(tablas)}")
    print("  Tablas encontradas:", ", ".join(tablas[:5]), 
          f"...y {len(tablas)-5} más" if len(tablas) > 5 else "")
else:
    print(f"❌ Error: {result.get('error')}")

# 8. TEST DE QUERY EN LENGUAJE NATURAL
print("\n8. 🗣️ TEST DE CONSULTA NATURAL:")
result = db_agent.run({
    "prompt": "¿Cuántos productos hay en cada categoría?",
    "connection": test_connection
})

if "error" not in result:
    query = result.get('query', 'N/A')
    results = result.get('results', [])
    
    print(f"✅ Query generada:\n  {query}")
    if results:
        print("✅ Resultados encontrados:")
        for i, row in enumerate(results[:3], 1):
            print(f"  Fila {i}: {row}")
        if len(results) > 3:
            print(f"  ... y {len(results)-3} más")
    else:
        print("⚠️ Sin resultados")
else:
    print(f"❌ Error: {result.get('error')}")

# 9. GUARDAR CONFIGURACIÓN PARA REUTILIZAR
print("\n9. 💾 GUARDANDO CONFIGURACIÓN DE TEST:")
test_config = {
    "project_root": str(project_root),
    "mysql_config": {
        "host": test_connection["host"],
        "port": test_connection["port"],
        "user": test_connection["user"],
        "database": test_connection["database"]
    },
    "system_info": system_info['environment'] if 'system_info' in locals() else {},
    "tables_available": tablas if 'tablas' in locals() else [],
    "provider_status": manager.get_current_provider_name() if 'manager' in locals() else None
}

# Guardar en archivo JSON
config_file = project_root / "test_config.json"
with open(config_file, 'w') as f:
    json.dump(test_config, f, indent=2)
print(f"✅ Configuración guardada en: {config_file}")

print("\n" + "="*50)
print("TEST INTEGRAL COMPLETADO")
print("="*50)

=== TEST INTEGRAL SISTEMA BODEGA ===

🔍 Ubicación: /home/dioni/Escritorio/agentes/bodega_app/notebooks/bodega_test
📂 Proyecto raíz: /home/dioni/Escritorio/agentes/bodega_app
✅ Path añadido a Python

2. 📁 VERIFICACIÓN DE ESTRUCTURA:

▸ Configuración:
  ✅ .env
  ✅ config/settings.yaml

▸ Utilidades:
  ✅ utils/env_loader.py
  ✅ utils/db_system_detector.py

▸ Proveedores:
  ✅ providers/provider_manager.py
  ✅ providers/base_provider.py
  ✅ providers/llm/openai_provider.py
  ✅ providers/llm/claude_provider.py

▸ Agentes:
  ✅ agent_modules/base_agent.py
  ✅ agent_modules/db_agent/agent.py

✅ ESTRUCTURA COMPLETA

3. 🔐 CARGA DE VARIABLES DE ENTORNO:

▸ Configuración MySQL:
  ✓ MYSQL_HOST: 127.0.0.1
  ✓ MYSQL_PORT: 3307
  ✓ MYSQL_USER: root
  ✓ MYSQL_DATABASE: bodega
  ✓ MYSQL_PASSWORD: ********

4. 🚀 INICIALIZACIÓN DEL SISTEMA:
✔️ ProviderManager importado correctamente
✅ ProviderManager: OpenAIProvider
🔍 Iniciando detección del sistema...
🔍 Sistema detectado exitosamente
✅ DBAgent creado

5. 

In [2]:
# Celda 2: Análisis Detallado de Base de Datos - Función Compacta

def analizar_base_datos(db_agent, connection_config):
    """Análisis completo y compacto de la base de datos"""
    
    print("=== ANÁLISIS DETALLADO DE BASE DE DATOS ===\n")
    
    # 1. OVERVIEW GENERAL
    print("1. 📊 OVERVIEW GENERAL:")
    queries = {
        "Información general": "SELECT DATABASE() as database_actual, COUNT(*) as total_tablas FROM information_schema.tables WHERE table_schema = 'bodega'",
        "Total productos": "SELECT COUNT(*) as total FROM productos",
        "Valor inventario": "SELECT SUM(stock_actual * precio_venta) as valor_total FROM productos WHERE activo = 1",
        "Stock crítico": "SELECT COUNT(*) as productos_criticos FROM productos WHERE stock_actual < stock_minimo"
    }
    
    for query_name, query in queries.items():
        result = db_agent.run({"prompt": query, "connection": connection_config})
        if "error" not in result:
            value = result.get('results', [[None]])[0][0]
            print(f"  ▸ {query_name}: {value}")
    
    # 2. ESTRUCTURA DE TABLAS PRINCIPALES
    print("\n2. 📋 ESTRUCTURA DE TABLAS PRINCIPALES:")
    tablas_importantes = ['productos', 'categorias', 'ventas', 'ubicaciones']
    
    for tabla in tablas_importantes:
        result = db_agent.run({"prompt": f"DESCRIBE {tabla}", "connection": connection_config})
        if "error" not in result:
            columns = result.get('results', [])
            print(f"\n  ▸ {tabla}:")
            print(f"    Campos: {len(columns)}")
            print(f"    Estructura: {', '.join([col[0] for col in columns[:5]])}")
            
            # Info adicional
            count_result = db_agent.run({"prompt": f"SELECT COUNT(*) FROM {tabla}", "connection": connection_config})
            if "error" not in count_result:
                count = count_result.get('results', [[0]])[0][0]
                print(f"    Registros: {count}")
    
    # 3. RELACIONES ENTRE TABLAS
    print("\n3. 🔗 RELACIONES IMPORTANTES:")
    print("  ▸ productos → categorias (categoria_id)")
    print("  ▸ productos → proveedores (proveedor_id)")
    print("  ▸ productos → ubicaciones (ubicacion_id)")
    print("  ▸ ventas → clientes (cliente_id)")
    print("  ▸ compras → proveedores (proveedor_id)")
    
    # 4. CONSULTAS OPERACIONALES CLAVE
    print("\n4. 💼 CONSULTAS OPERACIONALES CLAVE:")
    
    # Consulta 1: Productos más vendidos
    query_popular = """
    SELECT p.codigo, p.nombre, SUM(dv.cantidad) as total_vendido 
    FROM productos p 
    JOIN detalles_ventas dv ON p.producto_id = dv.producto_id 
    GROUP BY p.producto_id 
    ORDER BY total_vendido DESC 
    LIMIT 3
    """
    result = db_agent.run({"prompt": query_popular, "connection": connection_config})
    if "error" not in result:
        print("  ▸ Top 3 productos más vendidos:")
        for prod in result.get('results', []):
            print(f"    - {prod[0]}: {prod[1]} (Vendidos: {prod[2]})")
    
    # Consulta 2: Categorías con más stock
    query_stock = """
    SELECT c.nombre, SUM(p.stock_actual) as stock_total 
    FROM categorias c 
    JOIN productos p ON c.categoria_id = p.categoria_id 
    GROUP BY c.categoria_id 
    ORDER BY stock_total DESC
    """
    result = db_agent.run({"prompt": query_stock, "connection": connection_config})
    if "error" not in result:
        print("\n  ▸ Stock por categoría:")
        for cat in result.get('results', []):
            print(f"    - {cat[0]}: {cat[1]} unidades")
    
    # 5. EXPORTAR METADATA DEL SCHEMA
    print("\n5. 💾 GENERANDO METADATA DEL SCHEMA:")
    metadata = {
        "tablas": {},
        "relaciones": [
            {"tabla": "productos", "columna": "categoria_id", "referencia": "categorias"},
            {"tabla": "productos", "columna": "proveedor_id", "referencia": "proveedores"},
            {"tabla": "productos", "columna": "ubicacion_id", "referencia": "ubicaciones"},
            {"tabla": "ventas", "columna": "cliente_id", "referencia": "clientes"},
            {"tabla": "compras", "columna": "proveedor_id", "referencia": "proveedores"}
        ]
    }
    
    # Guardar metadata
    metadata_file = project_root / "bodega_schema_metadata.json"
    with open(metadata_file, 'w') as f:
        json.dump(metadata, f, indent=2)
    print(f"✅ Metadata guardada en: {metadata_file}")
    
    return metadata

# Ejecutar análisis
if 'db_agent' in globals() and 'test_connection' in globals():
    metadata = analizar_base_datos(db_agent, test_connection)
    print("\n✨ ANÁLISIS COMPLETO")
else:
    print("⚠️ Primero ejecuta la Celda 1 para configurar el entorno")

=== ANÁLISIS DETALLADO DE BASE DE DATOS ===

1. 📊 OVERVIEW GENERAL:
✅ Reutilizando conexión existente a mysql
  ▸ Información general: bodega
✅ Reutilizando conexión existente a mysql
  ▸ Total productos: 5
✅ Reutilizando conexión existente a mysql
  ▸ Valor inventario: 203000.00
✅ Reutilizando conexión existente a mysql
  ▸ Stock crítico: 0

2. 📋 ESTRUCTURA DE TABLAS PRINCIPALES:
✅ Reutilizando conexión existente a mysql

  ▸ productos:
    Campos: 15
    Estructura: producto_id, codigo, nombre, descripcion, categoria_id
✅ Reutilizando conexión existente a mysql
    Registros: 5
✅ Reutilizando conexión existente a mysql

  ▸ categorias:
    Campos: 5
    Estructura: categoria_id, nombre, descripcion, activo, fecha_creacion
✅ Reutilizando conexión existente a mysql
    Registros: 6
✅ Reutilizando conexión existente a mysql

  ▸ ventas:
    Campos: 8
    Estructura: venta_id, cliente_id, empleado_id, fecha_venta, total
✅ Reutilizando conexión existente a mysql
    Registros: 5
✅ Reutili

In [3]:
# Celda 1: Test Dinámico para Cualquier Base de Datos (CORREGIDA)

import os
import sys
from pathlib import Path
import json
from datetime import datetime  # Añadido import faltante
from pprint import pprint

# 1. CONFIGURACIÓN INICIAL
print("=== TEST DINÁMICO DE BASE DE DATOS ===\n")

# Detectar ubicación
current = Path().resolve()
project_root = current.parent.parent if current.name == "bodega_test" else Path().resolve()
print(f"📂 Proyecto raíz: {project_root}")

# Configurar Python path
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# 2. CARGAR VARIABLES DE ENTORNO Y DETERMINAR BD
from dotenv import load_dotenv
env_path = project_root / ".env"
load_dotenv(env_path)

# Detectar que BD está configurada
db_name = os.getenv('MYSQL_DATABASE', 'db_default')
print(f"\n🎯 Base de datos objetivo: {db_name}")

# 3. INICIALIZAR SISTEMA
from providers.provider_manager import ProviderManager
from agent_modules.db_agent.agent import DBAgent

config_path = project_root / "config" / "settings.yaml"
manager = ProviderManager(str(config_path))
db_agent = DBAgent(provider_manager=manager)

# 4. CONECTAR A LA BASE DE DATOS
test_connection = {
    "type": "mysql",
    "host": os.getenv('MYSQL_HOST', '127.0.0.1'),
    "port": int(os.getenv('MYSQL_PORT', 3307)),
    "user": os.getenv('MYSQL_USER', 'root'),
    "password": os.getenv('MYSQL_PASSWORD', ''),
    "database": db_name
}

# 5. VERIFICAR SI LA BASE DE DATOS EXISTE
print("\n🔎 Verificando existencia de la base de datos...")
check_db_result = db_agent.run({
    "prompt": f"SELECT SCHEMA_NAME FROM information_schema.SCHEMATA WHERE SCHEMA_NAME = '{db_name}'",
    "connection": {**test_connection, "database": None}  # Conectar sin db específica
})

if check_db_result.get('results'):
    print(f"✅ Base de datos '{db_name}' existe")
    
    # Conectar a la base específica
    connect_result = db_agent.run({
        "prompt": "SELECT DATABASE() as current_db",
        "connection": test_connection
    })
    
    if "error" not in connect_result:
        current_db = connect_result.get('results', [[None]])[0][0]
        print(f"✅ Conectado a: {current_db}")
        
        # 6. LISTAR TODAS LAS TABLAS
        tables_result = db_agent.run({
            "prompt": "SHOW TABLES",
            "connection": test_connection
        })
        
        if "error" not in tables_result:
            tables = [t[0] for t in tables_result.get('results', [])]
            total_tables = len(tables)
            print(f"\n📊 Total de tablas: {total_tables}")
            
            # Estrategia: Si hay pocas tablas (≤5), analizarlas en detalle
            # Si hay muchas, solo listarlas
            if total_tables <= 5:
                print(f"▸ Como hay ≤5 tablas, analizaré cada una en detalle:")
                analyze_in_detail = True
            else:
                print(f"▸ Como hay {total_tables} tablas, solo las listaré:")
                print("  Tablas encontradas:", ", ".join(tables))
                analyze_in_detail = False
            
            # 7. ANÁLISIS DINÁMICO DE TABLAS
            table_analysis = {}
            
            for i, table in enumerate(tables):
                print(f"\n{'='*60}")
                print(f"Tabla {i+1}/{total_tables}: {table}")
                print('='*60)
                
                if analyze_in_detail:
                    # Obtener estructura
                    structure_result = db_agent.run({
                        "prompt": f"DESCRIBE {table}",
                        "connection": test_connection
                    })
                    
                    if "error" not in structure_result:
                        columns = structure_result.get('results', [])
                        print(f"\n📋 Estructura ({len(columns)} campos):")
                        print("| Campo           | Tipo            | Null | Key |")
                        print("|-----------------|-----------------|------|-----|")
                        for col in columns:
                            print(f"| {col[0]:15} | {col[1]:15} | {col[2]:4} | {col[3]:3} |")
                        
                        # Contar registros
                        count_result = db_agent.run({
                            "prompt": f"SELECT COUNT(*) as total FROM {table}",
                            "connection": test_connection
                        })
                        
                        if "error" not in count_result:
                            total_records = count_result.get('results', [[0]])[0][0]
                            print(f"\n📦 Total de registros: {total_records}")
                            
                            # Mostrar primeros 5 registros si existen
                            if total_records > 0:
                                sample_result = db_agent.run({
                                    "prompt": f"SELECT * FROM {table} LIMIT 5",
                                    "connection": test_connection
                                })
                                
                                if "error" not in sample_result:
                                    samples = sample_result.get('results', [])
                                    print("\n🔍 Muestra de datos (primeros 5 registros):")
                                    for idx, sample in enumerate(samples, 1):
                                        print(f"  [{idx}] {sample}")
                            else:
                                print("⚠️ Tabla vacía")
                    
                    # Guardar análisis
                    table_analysis[table] = {
                        "columns": len(columns) if 'columns' in locals() else 0,
                        "records": total_records if 'total_records' in locals() else 0,
                        "status": "populated" if 'total_records' in locals() and total_records > 0 else "empty"
                    }
                else:
                    # Solo contar registros para tablas numerosas
                    count_result = db_agent.run({
                        "prompt": f"SELECT COUNT(*) as total FROM {table}",
                        "connection": test_connection
                    })
                    
                    if "error" not in count_result:
                        total_records = count_result.get('results', [[0]])[0][0]
                        status = "con datos" if total_records > 0 else "vacía"
                        print(f"  Estado: {status} ({total_records} registros)")
                    
                    table_analysis[table] = {
                        "records": total_records if 'total_records' in locals() else 0
                    }
            
            # 8. RESUMEN FINAL
            print("\n" + "="*60)
            print("RESUMEN DE LA BASE DE DATOS")
            print("="*60)
            print(f"Nombre: {db_name}")
            print(f"Total de tablas: {total_tables}")
            
            # Categorizar tablas
            empty_tables = [t for t, a in table_analysis.items() if a.get('records', 0) == 0]
            populated_tables = [t for t, a in table_analysis.items() if a.get('records', 0) > 0]
            
            print(f"Tablas vacías: {len(empty_tables)}")
            if empty_tables:
                print(f"  - {', '.join(empty_tables)}")
            
            print(f"Tablas con datos: {len(populated_tables)}")
            if populated_tables:
                print(f"  - {', '.join(populated_tables)}")
            
            # Guardar configuración global
            config_data = {
                "database": db_name,
                "timestamp": str(datetime.now()),  # Ahora datetime está importado
                "tables": table_analysis,
                "connection_info": {
                    "host": test_connection["host"],
                    "port": test_connection["port"],
                    "user": test_connection["user"]
                }
            }
            
            config_file = project_root / f"{db_name}_analysis.json"
            with open(config_file, 'w') as f:
                json.dump(config_data, f, indent=2)
            print(f"\n✅ Análisis guardado en: {config_file}")
        else:
            print(f"❌ Error al listar tablas: {tables_result.get('error')}")

=== TEST DINÁMICO DE BASE DE DATOS ===

📂 Proyecto raíz: /home/dioni/Escritorio/agentes/bodega_app

🎯 Base de datos objetivo: bodega
✔️ ProviderManager importado correctamente
🔍 Iniciando detección del sistema...
🔍 Sistema detectado exitosamente

🔎 Verificando existencia de la base de datos...
🔍 Intentando conexión con parámetros personalizados
🔌 Intentando conectar a mysql con parámetros: {'type': 'mysql', 'host': '127.0.0.1', 'port': 3307, 'user': 'root', 'password': '********', 'database': None}
🔌 Conectando a mysql con parámetros: {'type': 'mysql', 'host': '127.0.0.1', 'port': 3307, 'user': 'root', 'password': '********', 'database': None}
❌ Error al ejecutar consulta en mysql: 1046 (3D000): No database selected
✅ Base de datos 'bodega' existe
✅ Reutilizando conexión existente a mysql
✅ Conectado a: None
✅ Reutilizando conexión existente a mysql
❌ Error al ejecutar consulta en mysql: 1046 (3D000): No database selected

📊 Total de tablas: 0
▸ Como hay ≤5 tablas, analizaré cada una e